Option pricing is a fundamental problem in financial markets, where it is crucial to determine the fair price of financial derivatives. Traditional methods, such as the Black-Scholes model and Monte Carlo simulations, can be computationally expensive, especially for complex derivatives or when high precision is required. Quantum computing offers the potential to significantly speed up these calculations using algorithms like Quantum Amplitude Estimation (QAE) and its iterative version, IQAE.

An option is a financial contract that gives the holder the right, but not the obligation, to buy or sell an underlying asset at a specified price (strike price) on or before a specified date (expiry date). The two main types of options are:

 Gives the holder the right to buy the underlying asset.

 Gives the holder the right to sell the underlying asset.


The price of an option is influenced by various factors, including the current price of the underlying asset, the strike price, the time to maturity, volatility, and interest rates.



For a European call option, the payoff function at maturity \( T \) is given by:
\[
Payoff = $\max(S_T - K, 0)$
\]
where \( S_T \) is the price of the underlying asset at maturity and \( K \) is the strike price.


In financial modeling, it is common to assume that the underlying asset price follows a geometric Brownian motion, leading to a log-normal distribution of the asset price at maturity:
\[
S_T = S_0 $\exp\left((\mu - \frac{1}{2} \sigma^2) T + \sigma W_T \right)$
\]
where:

\( S_0 \) is the initial asset price,

\[$\ \mu $\] is the drift rate,
  
 \[$\ ( W_T ) $\] is a standard Brownian motion at time \[$\\( T ) $\].

\[$\ \sigma $\] is the volatility,

   




QAE is a quantum algorithm that provides a quadratic speedup over classical Monte Carlo methods for estimating the mean of a probability distribution. Given a quantum state $( |\psi\rangle )$ and an operator \( A \) such that
\[
A|0$\rangle = \sqrt{p}|\psi_1\rangle + \sqrt{1 - p}|\psi_0\rangle$,
\]
QAE estimates the amplitude \( p \) with high precision.



IQAE is an improved version of QAE that iteratively refines the estimate of the amplitude \( p \). It avoids the need for quantum phase estimation, making it more practical for near-term quantum devices.

 Prepare the initial quantum state representing the probability distribution of the underlying asset prices.
Define the oracle that marks the states corresponding to the desired payoff and construct the Grover operator.
 Iteratively apply the Grover operator and measure the state to refine the estimate of the amplitude \( p \).
 Use classical computation to process the measurement outcomes and update the amplitude estimate.




Prepare the quantum state that encodes the log-normal distribution of asset prices at maturity.

Encode the payoff function as a quantum oracle that marks the states where the payoff is non-zero.

Construct the Grover operator $( G = UWU^\dagger W^\dagger ), where ( U )$ is the unitary operator representing the oracle, and \( W \) is the diffusion operator.


Apply the Grover operator iteratively to amplify the amplitude of the marked states and estimate the amplitude \( p \) corresponding to the expected payoff.

Use quantum circuits to represent the log-normal distribution of asset prices.
Implement the payoff function as a quantum circuit.
 Use IQAE to estimate the expected payoff, which corresponds to the fair price of the option.



First, we import all necessary libraries and modules. We also define the parameters for the log-normal distribution and the European call option.


The log_normal_model_input.LogNormalModelInput class is used to define the parameters for the log-normal distribution. This includes the number of qubits, the mean (mu), and the standard deviation (sigma).


The function_input.FunctionCondition and function_input.FinanceFunctionInput classes are used to define the condition and parameters for the European call option.


A quantum function main is defined to calculate the payoff. This function allocates the necessary quantum resources and applies a simple calculation (ind *= x ** 2) to the allocated registers.


The create_model function is used to create a quantum model for the payoff function. The write_qmod function writes this model to a file for further use.


Several quantum functions are defined to implement Grover's algorithm components, including reflection_about_zero, my_diffuser, arith_equation, arith_oracle, and my_grover_operator.


The main quantum function main is defined to set up the quantum phase estimation (QPE) process using Grover's algorithm. This function allocates the necessary quantum registers and applies the QPE algorithm.


The create_model function is used again to create the main quantum model for amplitude estimation. Constraints are set to limit the maximum width of the quantum circuit.


The synthesize function is used to synthesize the quantum program from the defined quantum model. The show function displays the synthesized quantum circuit.


A classical execution function cmain is defined to incorporate Iterative Quantum Amplitude Estimation (IQAE). This function uses the built-in IQAE method to estimate the amplitude with a specified accuracy (epsilon) and confidence level (alpha).


The IQAE method is incorporated into the quantum program by creating a new quantum model that includes the IQAE classical execution function. This model is synthesized and the resulting quantum program is displayed.


A helper function synthesize_and_execute is defined to synthesize and execute the quantum program. This function also processes the results to extract the IQAE estimation value.

Finally, the results of the quantum program execution are checked and printed. If the results are not empty, the estimation value is printed.

In [ ]:
from classiq.applications.finance import log_normal_model_input, function_input
from classiq import execute, write_qmod
from classiq.execution import ExecutionPreferences

from classiq import (
    Output,
    QArray,
    QBit,
    QNum,
    allocate,
    allocate_num,
    create_model,
    qfunc,
    synthesize,
    write_qmod,
    QuantumProgram,
    show,
    execute,
    Constraints,
    cfunc,
    QCallable,
    set_constraints,
)

import numpy as np

# Parameters for log-normal distribution
num_qubits = 5
mu = 0.7
sigma = 0.13

log_normal_model = log_normal_model_input.LogNormalModelInput(
    num_qubits=num_qubits, mu=mu, sigma=sigma
)

# Parameters for European Call option
threshold = 1.9

condition = function_input.FunctionCondition(threshold=threshold, larger=True)
finance_function = function_input.FinanceFunctionInput(
    f="european call option",
    condition=condition,
)

# Quantum function for payoff calculation
VAR_SIZE = 4

@qfunc
def main(x: Output[QNum], ind: Output[QNum]) -> None:
    allocate_num(VAR_SIZE, False, VAR_SIZE, x)
    allocate(1, ind)

    ind *= x ** 2

# Create quantum model for payoff function
qmod_payoff = create_model(main)
write_qmod(qmod_payoff, "payoff_function_example")

# Define quantum functions for Grover's algorithm components
@qfunc
def reflection_about_zero(x: QArray[QBit]):
    lsb = QBit("lsb")
    msbs = QArray("msbs", QBit, x.len - 1)

    apply_to_all(X, x)
    bind(x, [msbs, lsb])
    control(msbs, lambda: Z(lsb))
    bind([msbs, lsb], x)
    apply_to_all(X, x)

@qfunc
def my_diffuser(sp_operand: QCallable[QArray[QBit]], x: QArray[QBit]):
    within_apply(
        lambda: invert(lambda: sp_operand(x)),
        lambda: reflection_about_zero(x),
    )

sp_oracle = lambda x: hadamard_transform(x)

@qfunc
def arith_equation(a: QNum, b: QNum, res: QBit):
    res ^= a + b <= 2

@qfunc
def arith_oracle(a: QNum, b: QNum):
    aux = QBit("aux")
    within_apply(
        lambda: (allocate(1, aux), X(aux), H(aux)), lambda: arith_equation(a, b, aux)
    )

@qfunc
def my_grover_operator(
    oracle_operand: QCallable[QArray[QBit]],
    sp_operand: QCallable[QArray[QBit]],
    x: QArray[QBit],
):
    oracle_operand(x)
    my_diffuser(sp_operand, x)
    U(0, 0, 0, np.pi, x[0])

NUM_PHASE_QUBITS = 5
A_SIZE = 2
B_SIZE = 2
DOMAIN_SIZE = A_SIZE + B_SIZE

@qfunc
def main(
    phase_reg: Output[QNum],
) -> None:
    state_reg = QArray("state_reg")
    allocate(DOMAIN_SIZE, state_reg)
    allocate_num(NUM_PHASE_QUBITS, False, NUM_PHASE_QUBITS, phase_reg)
    sp_oracle(state_reg)
    qpe(
        unitary=lambda: my_grover_operator(
            lambda x: arith_oracle(x[0:A_SIZE], x[A_SIZE: x.len]),
            sp_oracle,
            state_reg,
        ),
        phase=phase_reg,
    )

# Create the main quantum model for amplitude estimation
qmod = create_model(main)
qmod = set_constraints(qmod, constraints=Constraints(max_width=20))
write_qmod(qmod, "option_pricing")

# Synthesize the quantum program
qprog = synthesize(qmod)

# Show the synthesized circuit
show(qprog)

# Define IQAE classical execution function
DOMAIN_SIZE_QCONST = QConstant("DOMAIN_SIZE_QCONST", int, DOMAIN_SIZE)

@cfunc
def cmain():
    from classiq.qmod.builtins.classical_execution_primitives import iqae, save

    iqae_res = iqae(epsilon=1 / ((2 ** DOMAIN_SIZE_QCONST) * 2), alpha=0.01)
    save({"iqae_res": iqae_res})

# Incorporate IQAE into the quantum program
qmod_iqae = create_model(
    main,
    constraints=Constraints(optimization_parameter="width"),
    classical_execution_function=cmain,
)

write_qmod(qmod_iqae, "quantum_counting_iqae")
qprog_iqae = synthesize(qmod_iqae)
show(qprog_iqae)
from classiq import execute
def synthesize_and_execute(post_process):
    constraints = Constraints(max_width=25)
    qmod = create_model(
        get_main(),
        constraints=constraints,
        classical_execution_function=cmain,
    )
    write_qmod(qmod_iqae, "quantum_counting_iqae")
    print("Starting synthesis")
    qprog = synthesize(qmod)
    show(qprog)
    print("Starting execution")
    res = execute(qprog).result()
    iqae_res = res[0].value
    parsed_res = post_process(res[0].value)

    return (qmod, qprog, iqae_res, parsed_res)

if results:
    # Print the estimation value if available
    estimation_value = results[0].value  # Assuming estimation is the first result
    print(f"estimation : {estimation_value}")
else:
    print("No results found or empty result set.")


Opening: https://platform.classiq.io/circuit/e1427c10-a9da-464d-8857-951c709ba176?version=0.42.2
Opening: https://platform.classiq.io/circuit/e97cfb5f-0d52-41d6-a476-4f1e02e7b9ba?version=0.42.2
estimation : vendor_format_result={} counts={'10011': 1, '01111': 2, '11100': 16, '10101': 7, '10010': 2, '01101': 5, '00001': 2, '00110': 117, '01100': 2, '01000': 38, '11000': 25, '11010': 118, '01001': 16, '00101': 28, '11111': 3, '11011': 31, '10000': 4, '00011': 7, '00111': 765, '10110': 7, '11110': 4, '00100': 11, '00010': 8, '11001': 796, '00000': 5, '01110': 2, '10111': 14, '10100': 3, '01011': 3, '01010': 3, '11101': 3} counts_lsb_right=True parsed_states={'10011': {'phase_reg': 0.59375}, '01111': {'phase_reg': 0.46875}, '11100': {'phase_reg': 0.875}, '10101': {'phase_reg': 0.65625}, '10010': {'phase_reg': 0.5625}, '01101': {'phase_reg': 0.40625}, '00001': {'phase_reg': 0.03125}, '00110': {'phase_reg': 0.1875}, '01100': {'phase_reg': 0.375}, '01000': {'phase_reg': 0.25}, '11000': {'phas